In [1]:
import os.path as op
import matplotlib.pyplot as plt
from itertools import compress
import time
import logging
import mne
from mne import events_from_annotations
from collections import OrderedDict
import seaborn as sns
import pandas as pd
import pingouin as pg


import numpy as np
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import matplotlib
from itertools import compress
import mne
import mne_nirs
from mne.preprocessing.nirs import optical_density, beer_lambert_law

from mne_nirs.signal_enhancement import enhance_negative_correlation, short_channel_regression

from mne.preprocessing.nirs import (optical_density,
                                    temporal_derivative_distribution_repair,
                                    scalp_coupling_index)

from mne.stats import permutation_t_test

import ROI
from ROI import different_hb, different_roi
from meta import *
from filter_params import FILTER_DICT
from functions_fnirs import *

def fast_scanfiles(dirname, contains=None):
    subfiles = [f.path for f in os.scandir(dirname) if f.is_file()]
    if contains != None:
        subfiles = [i for i in subfiles if contains in i ]
    return subfiles

%gui qt
# from surfer import Brain

SFREQ = 2

In [2]:
%matplotlib auto


Using matplotlib backend: <object object at 0x7f0c200e5be0>


In [3]:
from matplotlib.colors import LinearSegmentedColormap

fnirs_colors = dict(
    
    hbo = '#C91111',
    hbr = '#005D7E',
    
    hbt= '#A4C210',
#     hbr='#135181',
  )



# penfield_colors = dict(
#     M1 = '#C86494',
#     S1 = '#827FA9',
#     between = '#A4C210'
# )

def rgb(hex_color, alpha):
    """Converts a hex color code to RGBA format with specified alpha value."""
    # Remove '#' from hex color code and convert to integer
    hex_color = hex_color.lstrip('#')
    rgb = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
    # Convert RGB to RGBA format with specified alpha value
    rgba = (rgb[0] / 255, rgb[1] / 255, rgb[2] / 255, alpha)
    return rgba

alpha=0.5
transparent_blue, blue = rgb(fnirs_colors['hbr'], alpha), rgb(fnirs_colors['hbr'], 0.95)
transparent_red, red = rgb(fnirs_colors['hbo'], alpha), rgb(fnirs_colors['hbo'], 0.95)
desatured_blue = '#294E6B'
desatured_red = '#A41919'


legend_params_dict = dict(
                            loc='center', 
                            fontsize=14,       
                            borderpad=1.5, 
                            labelspacing=1.5,
                            markerscale=10, 
                            framealpha=1
)

curves_subplot_params = dict(
 top=0.92,
 bottom=0.11,
 left=0.065,
 right=0.945,
 hspace=0.2,
 wspace=0.35
)

two_topomaps_subplot_params = dict(top=0.89,
bottom=0.21,
left=0.115,
right=0.98,
hspace=0.175,
wspace=0.0)

one_topomap_subplot_params = dict(top=0.92,
bottom=0.075,
left=0.105,
right=0.98,
hspace=0.105,
wspace=0.0)


mask_params = dict(marker='o', 
                   markerfacecolor='black', 
                   markeredgecolor='white',
                   linewidth=0, 
                   markersize=12
                   )

color_list = [(0, fnirs_colors['hbr']), 
              (0.5, 'white'), 
              (1, fnirs_colors['hbo'])]
custom_cmap = LinearSegmentedColormap.from_list('Custom_cmap', color_list, )


# color_list = [(0, desatured_blue), 
#               (0.5, 'white'), 
#               (1, desatured_red)]

sphere=(0.0, 0.03, 0.0, 0.095)

C3 = [  'S9_D18 hbo',
'S9_D13 hbo',
'S16_D18 hbo',
'S16_D13 hbo',
     ]
C4 = [
      'S12_D21 hbo',
'S12_D14 hbo',
      'S17_D14 hbo',
'S17_D21 hbo',
]

m1_group_left = [info_hbo_total.ch_names.index(ch) for ch in C3]
m1_group_right = [info_hbo_total.ch_names.index(ch) for ch in C4]
chanels = m1_group_left + m1_group_right
mask = np.zeros(102)
mask[chanels] = True

In [249]:
# path_to_haemo = '/home/avaldamon/Desktop/minmaxed/haemodynamics_np'
# topo_dir = '/home/avaldamon/Desktop/minmaxed/topomaps_np/'


topo_dir = '/mnt/diskus/hbo_pics/topomaps_rel_np/'
path_to_haemo = '/mnt/diskus/hbo_pics//relation_np/'
n_epochs = np.load('/mnt/diskus/hbo_pics//relation_np/AA_ME_rel_left.npy').shape[0]


# topo_dir = '/mnt/diskus/more_or_less_final/topomaps_rel_np/'
# path_to_haemo = '/mnt/diskus/more_or_less_final/relation_np'
# n_epochs = np.load('/mnt/diskus/more_or_less_final/relation_np/AA_ME_rel_left.npy').shape[0]


# new_shape = (13, 60, 33)
# exclude = ['AB', 'DT', 'EI', 'ET', 'IV', 'MM', 'OK']

exclude = ['VL', 'OK', 'DK', 'OP', 'AL']
# exclude = []
new_shape = (20-len(exclude), n_epochs, 33)
ids_of_subjs = sorted(list(set(ids_of_subjs) - set(exclude)))

# new_shape = (20, 35, 30)
# exclude = []

def concatenate_dicts(haemo_dict, new_shape=new_shape):
    new_dict = {}
    for key in haemo_dict.keys():
        a = np.vstack(haemo_dict[key])
        if new_shape:
            a = a.reshape(new_shape[0], new_shape[1], new_shape[2])
        new_dict[key] = a
    return new_dict

def make_subdicts(haemo_dict, contains):
    new_dict = {key[3:]:value for key, value in haemo_dict.items() if contains in key}
    return new_dict

def concatenate_dicts(haemo_dict, new_shape=new_shape):
    new_dict = {}
    for key in haemo_dict.keys():
        a = np.vstack(haemo_dict[key])
        if new_shape:
            a = a.reshape(new_shape[0], new_shape[1], new_shape[2])
        new_dict[key] = a
    return new_dict

def get_roi_evokeds(new_dict, key, slicer=[int(new_shape[1]/3), int(new_shape[1]/3*2)]):
    M1_epochs = np.median(new_dict[key][:, :slicer[0], :], axis=1)
    S1_epochs = np.median(new_dict[key][:, slicer[0]:slicer[1], :], axis=1)
    SMZ_epochs = np.median(new_dict[key][:, slicer[1]:, :], axis=1)
    M1 = np.mean(M1_epochs, axis=0) 
    S1 = np.mean(S1_epochs, axis=0)
    SMZ = np.mean(SMZ_epochs, axis=0) 
    
    return M1, S1, SMZ, M1_epochs, S1_epochs, SMZ_epochs


In [250]:
fnirs_dir = "/mnt/diskus/fNIRS data ME_MI_TS_TI_SA"
subfolders = sorted(fast_scandir(fnirs_dir)[20:])
new_list = [i.split('/')[-1] for i in subfolders]

In [251]:
def new_fast_scanfiles(dirname, contains=None, ex=[]):
    subfiles = [f.path for f in os.scandir(dirname) if f.is_file()]
    if contains != None:
        subfiles = [i for i in subfiles if contains in i ]
    if exclude != None:
        subfiles = [i for i in subfiles if i.split('/')[-1][0:2] not in exclude ]
    return subfiles


In [252]:
np.load('/mnt/diskus/hbo_pics/relation_np/LJ_TI_rel_left.npy').shape

(90, 33)

In [253]:
def fast_scanfiles(dirname, contains=None, exclude=[]):
    subfiles = [f.path for f in os.scandir(dirname) if f.is_file()]
    if contains != None:
        subfiles = [i for i in subfiles if contains in i ]
    if exclude != None:
        subfiles = [i for i in subfiles if i.split('/')[-1][0:2] not in exclude ]
    return subfiles

haemo_dict = dict(
    
ME_smr_left = [np.load(i) for i in sorted(new_fast_scanfiles(path_to_haemo,
                                        contains='ME_rel_left', ex=exclude))],
MI_smr_left = [np.load(i) for i in sorted(new_fast_scanfiles(path_to_haemo,
                                        contains='MI_rel_left', ex=exclude))],
TS_smr_left = [np.load(i) for i in sorted(new_fast_scanfiles(path_to_haemo,
                                        contains='TS_rel_left', ex=exclude))],
TI_smr_left = [np.load(i) for i in sorted(new_fast_scanfiles(path_to_haemo,
                                        contains='TI_rel_left', ex=exclude))],
SA_smr_left = [np.load(i) for i in sorted(new_fast_scanfiles(path_to_haemo,
                                        contains='SA_rel_left', ex=exclude))],
)

subj_n = len(haemo_dict['ME_smr_left'])


ME_subdict = make_subdicts(haemo_dict, contains='ME')
MI_subdict = make_subdicts(haemo_dict, contains='MI')
TS_subdict = make_subdicts(haemo_dict, contains='TS')
TI_subdict = make_subdicts(haemo_dict, contains='TI')
SA_subdict = make_subdicts(haemo_dict, contains='SA')

ME_subdict = concatenate_dicts(make_subdicts(haemo_dict, contains='ME'))
MI_subdict = concatenate_dicts(make_subdicts(haemo_dict, contains='MI'))
TS_subdict = concatenate_dicts(make_subdicts(haemo_dict, contains='TS'))
TI_subdict = concatenate_dicts(make_subdicts(haemo_dict, contains='TI'))
SA_subdict = concatenate_dicts(make_subdicts(haemo_dict, contains='SA'))


topo_dict = dict(

ME_topo = [np.load(i) for i in fast_scanfiles(topo_dir,
                                        contains='ME', exclude=exclude)],
MI_topo = [np.load(i) for i in fast_scanfiles(topo_dir,
                                        contains='MI', exclude=exclude)],
TS_topo = [np.load(i) for i in fast_scanfiles(topo_dir,
                                        contains='TS', exclude=exclude)],
TI_topo = [np.load(i) for i in fast_scanfiles(topo_dir,
                                        contains='TI', exclude=exclude)],
SA_topo = [np.load(i) for i in fast_scanfiles(topo_dir,
                                        contains='SA', exclude=exclude)],
)
huh = concatenate_dicts(topo_dict, new_shape=None)

info_hbr_total = mne.io.read_info("/mnt/diskus/infos/info_hbr_total_info.fif")
info_left_smz = mne.io.read_info("/mnt/diskus/infos/info_left_smz_info.fif")
info_right_smz = mne.io.read_info("/mnt/diskus/infos/info_right_smz_info.fif")
info_hbo_total = mne.io.read_info("/mnt/diskus/infos/info_hbo_total_info.fif")

In [254]:
from mne import stc_near_sensors, read_source_spaces
from mne.utils import get_subjects_dir
from mne.io.constants import FIFF

def plot_3d_evoked_array( ea, info=info_hbo_total,
                          value="Coef.",
                          background='w', figure=None, clim='auto',
                          mode='weighted', colormap='RdBu_r',
                          surface='pial', hemi='lh', size=800,
                          view=None, colorbar=True, distance=0.025,
                          subjects_dir=None, src=None, verbose=False,
                        new_shape = (102, 1)):
    ea = mne.EvokedArray(ea.reshape(new_shape), info=info_hbo_total)
    # TODO: mimic behaviour of other MNE-NIRS glm plotting options
    if subjects_dir is None:
        subjects_dir = get_subjects_dir(raise_error=True)
    if src is None:
        fname_src_fs = os.path.join(subjects_dir, 'fsaverage', 'bem',
                                    'fsaverage-ico-5-src.fif')
        src = read_source_spaces(fname_src_fs)

    for idx in range(len(ea.ch_names)):
        ea.info['chs'][idx]['coord_frame'] = FIFF.FIFFV_COORD_HEAD
        
    # Generate source estimate
    kwargs = dict(picks=[i for i in range(0, 102)],
        evoked=ea, subject='fsaverage', 
#                   trans='/home/avaldamon/Desktop/new_trans.fif',
                  trans='fsaverage',
        distance=distance, mode=mode, surface=surface,
        subjects_dir=subjects_dir, src=src, project=True)
    stc = stc_near_sensors(**kwargs, verbose=verbose)


    return stc

In [255]:
trans = mne.transforms._get_trans('fsaverage', "head", "mri", allow_none=True)[1]

In [256]:
sample_data_folder = mne.datasets.sample.data_path()
subjects_dir = sample_data_folder / "subjects"

In [257]:
# raw_fnirs = mne.io.read_raw_nirx('/mnt/diskus/fNIRS data ME_MI_TS_TI_SA/AB/AB_MI')
# mne.gui.coregistration(subject='sample', trans=trans, subjects_dir=subjects_dir)

In [258]:
# eeg_info = mne.io.read_info('/mnt/diskus/infos/eeg_3_info.fif')
# m1_info = mne.io.read_info('/mnt/diskus/infos/m1_info.fif')
# s1_info = mne.io.read_info('/mnt/diskus/infos/s1_info.fif')

# m1_info = mne.io.read_info('/mnt/diskus/infos/m1_alter_info.fif')
# s1_info = mne.io.read_info('/mnt/diskus/infos/s1_alter_info.fif')

m1_info = mne.io.read_info('/mnt/diskus/infos/M1_info.fif')
s1_info = mne.io.read_info('/mnt/diskus/infos/S1_info.fif')
between_info = mne.io.read_info('/mnt/diskus/infos/BETWEEN_info.fif')




In [259]:
%matplotlib auto

Using matplotlib backend: QtAgg


In [260]:
# import numpy as np
# from scipy.stats import ttest_rel

# def permutation_t_test_single_parameter(data1, data2, num_permutations, tail="two-sided"):
#     actual_statistics = np.zeros(data1.shape[1])
#     for i in range(data1.shape[1]):
#         if tail == "two-sided":
#             actual_statistic, _ = ttest_rel(data1[:, i], data2[:, i])
#         elif tail == "greater":
#             actual_statistic, _ = ttest_rel(data1[:, i], data2[:, i], alternative='greater')
#         elif tail == "less":
#             actual_statistic, _ = ttest_rel(data1[:, i], data2[:, i], alternative='less')
#         actual_statistics[i] = actual_statistic
#     combined_data = np.concatenate([data1, data2], axis=0)
#     n1 = len(data1)
#     permuted_stats = np.zeros((num_permutations, data1.shape[1]))
#     for i in range(num_permutations):
#         np.random.shuffle(combined_data)
#         perm_data1 = combined_data[:n1]
#         perm_data2 = combined_data[n1:]
#         for j in range(data1.shape[1]):
#             if tail == "two-sided":
#                 permuted_stat, _ = ttest_rel(perm_data1[:, j], perm_data2[:, j])
#             elif tail == "greater":
#                 permuted_stat, _ = ttest_rel(perm_data1[:, j], perm_data2[:, j], alternative='greater')
#             elif tail == "less":
#                 permuted_stat, _ = ttest_rel(perm_data1[:, j], perm_data2[:, j], alternative='less')
#             permuted_stats[i, j] = permuted_stat
#     p_values = np.zeros(data1.shape[1])
#     for j in range(data1.shape[1]):
#         if tail == "two-sided":
#             p_values[j] = np.sum(np.abs(permuted_stats[:, j]) >= np.abs(actual_statistics[j])) / num_permutations
#         elif tail == "greater":
#             p_values[j] = np.sum(permuted_stats[:, j] >= actual_statistics[j]) / num_permutations
#         elif tail == "less":
#             p_values[j] = np.sum(permuted_stats[:, j] <= actual_statistics[j]) / num_permutations
#     return p_values

# # Assuming ME, MI, TS, TI are numpy arrays of shapes (15, 102)
# ME, TS, MI, TI = topo_list[0], topo_list[1], topo_list[2], topo_list[3]
# num_permutations = 10000  # You can adjust the number of permutations

# ME_TS_parameter_p_values_greater = permutation_t_test_single_parameter(ME, TS, num_permutations, tail="greater")
# TS_ME_parameter_p_values_greater = permutation_t_test_single_parameter(TS, ME, num_permutations, tail="greater")

# # Perform permutation t-test between TI and MI
# MI_TI_parameter_p_values_greater = permutation_t_test_single_parameter(MI, TI, num_permutations, tail="greater")
# TI_MI_parameter_p_values_greater = permutation_t_test_single_parameter(TI, MI, num_permutations, tail="greater")

# confidence = 0.05

# mask_ME = ME_TS_parameter_p_values_greater < confidence
# mask_TS = TS_ME_parameter_p_values_greater < confidence
# mask_MI = MI_TI_parameter_p_values_greater < confidence
# mask_TI = TI_MI_parameter_p_values_greater < confidence

In [261]:
# fig, ax = plt.subplots(figsize=(14, 14))
# top_n_chans = 10


# ### LEFT SMR TOPO ###
# a = mne.viz.plot_sensors(
#                         info=info_hbo_total,
#                         axes=ax,
#                         kind='select',
#                         sphere=(0.0, 0.0185, 0.0, 0.085),

# #                             show=True,
# #                             names=info_hbo_total.ch_names
# )

# target_chans = ([-0.04930527091026306, 0.023039783409330994],
#                 [0.04507893744856119, 0.02398763585370034],
#                 [-0.0012414913401007652, 0.023633225545287132])
# for coord in target_chans:
#     ax.plot(*coord, marker='o', color='#717171', ms=15)

In [262]:
fig, ax = plt.subplots(figsize=(10, 10))
topo_linewidth = 2
pointsize = 60
topo_width = topo_height = '30%'
a = mne.viz.plot_sensors(
                        info_left_smz, 
                        ch_groups=[
                                        get_channel_indices(M1_LEFT_ROI_HBO, info=info_left_smz), 
                                        get_channel_indices(REMAINING_LEFT_ROI_HBO, info=info_left_smz),
                                        get_channel_indices(S1_LEFT_ROI_HBO, info=info_left_smz)
                        ],
                        sphere=(0.0, 0.0185, 0.0, 0.085),

                        axes=ax,
                        pointsize=200, 
                        linewidth=topo_linewidth,
    cmap=custom_cmap
)
target_chans = ([-0.04930527091026306, 0.023039783409330994],
                [0.04507893744856119, 0.02398763585370034],
                [-0.0012414913401007652, 0.023633225545287132])
for coord in target_chans:
    ax.plot(*coord, marker='o', color='#717171', ms=20)

In [263]:
m1_group_left = list(get_channel_indices(m1, info_hbo_total))
s1_group_left = list(get_channel_indices(s1, info_hbo_total))


In [300]:
curves_hb = 'hbo'
SUBJECT = 'AVERAGE'
CONDITION_LIST = ['ME', 'TS', 'MI', 
                  'TI', 
#                   'SA'
                 ]
TOPO_CONDITION_LIST = ['ME_topo', 'TS_topo', 'MI_topo', 
                       'TI_topo', 
#                        'SA_topo'
                      ]
subdicts = [ME_subdict, TS_subdict, MI_subdict, 
            TI_subdict, 
#             SA_subdict
           ]
storage = []

SFREQ = 2
TMIN, TMAX = -2, 14.5

holder = 0
func=ez_median
method='mean'
n_boot = 1000

df_initial = pd.DataFrame()
df_smz = pd.DataFrame()
stc_list = []
topo_data = []

m1_curves_data = []
s1_curves_data = []

topo_dict_data = dict(
ME=[], MI=[], TS=[], TI=[])
sign_sensors_data = dict(
ME=[], MI=[], TS=[], TI=[])

curves_M1_dict_data = dict(
ME=[], MI=[], TS=[], TI=[])
ci_epochs_M1_data = dict(
ME=[], MI=[], TS=[], TI=[])

curves_S1_dict_data = dict(
ME=[], MI=[], TS=[], TI=[])
ci_epochs_S1_data = dict(
ME=[], MI=[], TS=[], TI=[])

curves_SMA_dict_data = dict(
ME=[], MI=[], TS=[], TI=[]) 
ci_epochs_SMA_data = dict(
ME=[], MI=[], TS=[], TI=[])

screenshots_dict = dict(
ME=[], MI=[], TS=[], TI=[]) 

top_chans_M1_dict = dict(
ME=[], MI=[], TS=[], TI=[])
top_chans_S1_dict = dict(
ME=[], MI=[], TS=[], TI=[]) 

for i in range(len(subdicts)):
    CONDITION = CONDITION_LIST[i]
    subdict = subdicts[i]
    
    ####################################


    subdict_topo = huh[TOPO_CONDITION_LIST[i]]
    data = subdict_topo
    
    
#     if CONDITION == 'ME':
#         pick_mask = mask_ME
#     elif CONDITION == 'MI':
#         pick_mask = mask_MI
#     elif CONDITION == 'TS':
#         pick_mask = mask_TS
#     elif CONDITION == 'TI':
#         pick_mask = mask_TI
    
    n_permutations = 100000
    T0, p_values, H0 = permutation_t_test(data, n_permutations, n_jobs=-1)
    
#     bad_ch = np.array([25, 41, 42, 43, 46, 47, 56, 57, 58, 
#               59, 60, 61, 62, 63, 68, 69, 70, 71, 
#               80, 81, 82, 83, 84, 85, 91, 92, 93, 
#               94, 95, 99, 100, 101])
    
    pick_mask = p_values <= 0.001
#     pick_mask[bad_ch] = False
    

#     significant_sensors = info_hbo_total.ch_names[p_values <= 0.000005]
#     significant_sensors_names = [info_hbo_total.ch_names[k] for k in significant_sensors]

#     print("Number of significant sensors : %d" % len(significant_sensors))
#     print("Sensors names : %s" % significant_sensors_names)

    smr_in_peak_timestamp = np.mean(data, axis=0)[:102]
    
    stc = plot_3d_evoked_array(ea=smr_in_peak_timestamp)
    
    stc_list.append(stc)
    top_n_chans = 10

#     rel_mask_SMR, rel_top_dict_SMR = get_top_channels_mask(smr_in_peak_timestamp, 
#                                                     info_hbo_total, 
#                                                     top_n_chans)

    ###########################
    fig, ax = plt.subplots(figsize=(14, 14))
    ylims = [-5, 5]
    ### LEFT SMR TOPO ###
    a = mne.viz.plot_topomap(
                            data=smr_in_peak_timestamp,
                            pos=info_hbo_total,
                            vlim=ylims,
                            axes=ax,
                            contours=3,
                            extrapolate='local',
                            image_interp='linear',
#                             cmap=custom_cmap,
                            cmap='seismic',
                            sphere=(0.0, 0.0185, 0.0, 0.085),
#                             mask=rel_mask_SMR,
                            mask = pick_mask,
                            mask_params=mask_params,
#                             show=True,
#                             names=info_hbo_total.ch_names
    )
#     C3 = ['S9', 'D13', 'D18', 'S16']
#     C4 = ['D14', 'S12', 'S17', 'D21']
#     Cz = ['S10', 'S11', 'D19', 'D20']
#     chpos = info_hbo_total.get_montage().get_positions()['ch_pos']

#     C3_pos = np.mean([chpos[i] for i in C3], axis=0 )
#     C4_pos = np.mean([chpos[i] for i in C4], axis=0 )
#     Cz_pos = np.mean([chpos[i] for i in Cz], axis=0 )
    target_chans = ([-0.04930527091026306, 0.023039783409330994],
                    [0.04507893744856119, 0.02398763585370034],
                    [-0.0012414913401007652, 0.023633225545287132])
    for coord in target_chans:
        ax.plot(*coord, marker='o', color='#717171', ms=15)

    ### TITLES ###
    ax.set_title(
            f'Topography of {curves_hb} in subject {SUBJECT}\n in {CONDITION}  in time range 4:12 s',
    fontsize=18)
    
    holder = smr_in_peak_timestamp

    ### COLORBAR SETTINGS ###
    sm = plt.cm.ScalarMappable(
#                         cmap=custom_cmap,
                        cmap='seismic',
        
                        norm=matplotlib.colors.Normalize(vmin=ylims[0], vmax=ylims[1]))
    one_topomap_subplot_params = dict(top=0.92,
                                        bottom=0.075,
                                        left=0.105,
                                        right=0.98,
                                        hspace=0.20,
                                        wspace=0.0)
    cbaxes = fig.add_axes([0.075, 0.25, 0.03, 0.5]) # setup colorbar axes. 
    cbar = plt.colorbar(mappable=sm, cax=cbaxes, pad=0.15, orientation='vertical')
    cbar.set_label('Hb difference SMR-REST, Δ μM\L', loc='center', size=12)
    cbar.ax.yaxis.set_label_coords(-0.65, 0.5)
    topo_dict_data[CONDITION] = smr_in_peak_timestamp
    
    sign_sensors_data[CONDITION] = pick_mask
    source_spaces_dict_data[CONDITION] = stc
    
    plt.subplots_adjust(**one_topomap_subplot_params)
    
    fig.savefig(f'/home/avaldamon/Desktop/mean_pictures/delta/TOPO_DELTA_{CONDITION}.png')
    fig.clear()
    plt.close(fig)
    
    ##################################
    
        
    fig, ax = plt.subplots(figsize=(14, 14))
    ylims = (0.01, 0.075)
    ### LEFT SMR TOPO ###
    print(p_values)
    b = mne.viz.plot_topomap(
                            data=p_values,
                            pos=info_hbo_total,
                            vlim=ylims,
                            axes=ax,
                            contours=3,
                            extrapolate='local',
                            image_interp='linear',
#                             cmap=custom_cmap,
                            cmap='seismic',
                            sphere=(0.0, 0.0185, 0.0, 0.085),
#                             mask=rel_mask_SMR,
                            mask = pick_mask,
                            mask_params=mask_params,
#                             show=True,
#                             names=info_hbo_total.ch_names
    )
#     C3 = ['S9', 'D13', 'D18', 'S16']
#     C4 = ['D14', 'S12', 'S17', 'D21']
#     Cz = ['S10', 'S11', 'D19', 'D20']
#     chpos = info_hbo_total.get_montage().get_positions()['ch_pos']

#     C3_pos = np.mean([chpos[i] for i in C3], axis=0 )
#     C4_pos = np.mean([chpos[i] for i in C4], axis=0 )
#     Cz_pos = np.mean([chpos[i] for i in Cz], axis=0 )
    target_chans = ([-0.04930527091026306, 0.023039783409330994],
                    [0.04507893744856119, 0.02398763585370034],
                    [-0.0012414913401007652, 0.023633225545287132])
    for coord in target_chans:
        ax.plot(*coord, marker='o', color='#717171', ms=15)

    ### TITLES ###
    ax.set_title(
            f'Topography of PVALUES in subject {SUBJECT}\n in {CONDITION}  in time range 4:12 s',
    fontsize=18)
    
#     holder = smr_in_peak_timestamp

    ### COLORBAR SETTINGS ###
    sm = plt.cm.ScalarMappable(
#                         cmap=custom_cmap,
                        cmap='seismic',
        
                        norm=matplotlib.colors.Normalize(vmin=ylims[0], vmax=ylims[1]))
    one_topomap_subplot_params = dict(top=0.92,
                                        bottom=0.075,
                                        left=0.105,
                                        right=0.98,
                                        hspace=0.20,
                                        wspace=0.0)
    cbaxes = fig.add_axes([0.075, 0.25, 0.03, 0.5]) # setup colorbar axes. 
    cbar = plt.colorbar(mappable=sm, cax=cbaxes, pad=0.15, orientation='vertical')
    cbar.set_label('Pvalues, Δ μM\L', loc='center', size=12)
    cbar.ax.yaxis.set_label_coords(-0.65, 0.5)
    plt.subplots_adjust(**one_topomap_subplot_params)

    fig.savefig(f'/home/avaldamon/Desktop/mean_pictures/delta/PVALS_{CONDITION}.png')
    fig.clear()
    plt.close(fig)
    
    
    #############
    
    M1_evoked_SMR_left, S1_evoked_SMR_left, SMZ_evoked_SMR_left, \
    M1_epochs_SMR, S1_epochs_SMR, SMZ_epochs_SMR = get_roi_evokeds(subdict, key='smr_left')
    
#     m1 = [
#         'S9_D13 hbo', 
#           'S10_D13 hbo', 'S10_D16 hbo', 'S10_D19 hbo', 'S16_D13 hbo'
#     ]
    
#     s1 = [
#         'S16_D16 hbo', 
#         'S16_D18 hbo', 'S16_D19 hbo', 'S16_D23 hbo', 
#         'S24_D19 hbo']
    
    
    ### NEW MANUALLY SELECTED ###
    
    s1 =  ['S9_D18 hbo',
  'S13_D18 hbo',
  'S15_D12 hbo',
  'S16_D16 hbo',
  'S16_D18 hbo',
  'S16_D19 hbo']
    m1 = ['S9_D6 hbo',
  'S9_D10 hbo',
  'S9_D12 hbo',
  'S9_D13 hbo',
  'S10_D13 hbo',
  'S10_D16 hbo']
    between =  ['S10_D19 hbo', 'S13_D12 hbo', 'S16_D13 hbo']


    #BASED ON THE PERMUTATIONS
    
#     m1 =  ['S9_D13 hbo',
#       'S9_D18 hbo',
#       'S10_D13 hbo',
#       'S10_D16 hbo',
#       'S10_D19 hbo',
#       'S16_D13 hbo']
#     s1 = ['S13_D18 hbo',
#       'S16_D16 hbo',
#       'S16_D18 hbo',
#       'S16_D19 hbo',
#       'S16_D23 hbo',
#       'S24_D19 hbo']

    #### EMPIRICAL ####
#     m1 =  [
#         'S9_D13 hbo',
#       'S10_D13 hbo',
#       'S10_D16 hbo',
#       'S10_D19 hbo',
#       'S16_D13 hbo'
#           ]

#     s1 = [
#       'S9_D18 hbo',
#       'S16_D16 hbo',
#       'S16_D18 hbo',
#       'S16_D19 hbo',
#       'S16_D23 hbo',
#          ]

    ################################
    
    ### PLOT HEMODYNAMICS PER SE ###
    m1_group_left = list(get_channel_indices(m1, info_hbo_total))
    s1_group_left = list(get_channel_indices(s1, info_hbo_total))
    between_group_left = list(get_channel_indices(between, info_hbo_total))
    smz_group_left =  m1_group_left+s1_group_left+between_group_left

    times = np.arange(TMIN, TMAX, 1/SFREQ)
    tmin, tmax = TMIN, TMAX
    linewidth = 1.5
    ylims = (-2, 16)
#     ylims = (0.1, 0.8)
    topo_linewidth = 1
    pointsize = 20
    topo_width = topo_height = '30%'

    fig, axes = plt.subplots(1, 1, figsize=(14, 14))


    ### LEFT PART ###

    line_m1_smr, = axes.plot(times, M1_evoked_SMR_left, label=f'M1/{curves_hb} SMR', 
             color=penfield_colors['M1'])
    make_ci(ax=axes, to_plot_around=M1_evoked_SMR_left, 
            epochs_data=M1_epochs_SMR, 
            alpha=0.25,  color=penfield_colors['M1'], multiply=False, n_boot=n_boot,
           func=func, method=method)
    
    curves_M1_dict_data[CONDITION] = M1_evoked_SMR_left
    ci_epochs_M1_data[CONDITION] = M1_epochs_SMR

    line_s1_smr, = axes.plot(times, S1_evoked_SMR_left, label=f'S1/{curves_hb} SMR', 
             color=penfield_colors['S1'])
    make_ci(ax=axes, to_plot_around=S1_evoked_SMR_left, 
            epochs_data=S1_epochs_SMR, 
            alpha=0.25,  color=penfield_colors['S1'], multiply=False, n_boot=n_boot,
           func=func, method=method)
    
    curves_S1_dict_data[CONDITION] = S1_evoked_SMR_left
    ci_epochs_S1_data[CONDITION] = S1_epochs_SMR


    fill_1 = filler_between(axes, ylims)

    set_axis_properties(axes,
                        ylims=ylims, 
                        tlims=(tmin, tmax), 
                        title=f'HbO response in M1 and S1 in LEFT hemisphere\n {SUBJECT} in {subj_n} subjects, Condition {CONDITION}',
                        xlabel='Time, s', 
                        ylabel='Hb concentration, Δ μM\L',  
                        linewidth=2, 
                        fontsize=16, 
                        title_size=18,
                        legend_flag=False
                            )
    ### LEGENDS PARAMS ###
    ax_legend = fig.add_axes([0.75, 0.10, 0.25, 0.25])
    ax_legend.legend([line_m1_smr, 
                      line_s1_smr,
                      fill_1], 
                     [f'M1 {curves_hb} SMR', 
                      f'S1 {curves_hb} SMR', 
                     'Task duration'], 
                    **legend_params_dict) 

    ax_legend.set_frame_on(False)
    ax_legend.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
    
    plt.subplots_adjust(**curves_subplot_params)
    fig.savefig(f'/home/avaldamon/Desktop/mean_pictures/delta/HAEMO_{CONDITION}_M1S1.png')
    plt.close()
    fig.clear()
    
    
    
    fig, axes = plt.subplots(1, 1, figsize=(14, 14))


    ### LEFT PART ###

    line_smz_smr, = axes.plot(times, SMZ_evoked_SMR_left, label=f'M1/{curves_hb} SMR', 
             color=fnirs_colors['hbt'])
    make_ci(ax=axes, to_plot_around=SMZ_evoked_SMR_left, 
            epochs_data=SMZ_epochs_SMR, 
            alpha=0.25,  color=fnirs_colors['hbt'], multiply=False, n_boot=n_boot,
           func=func, method=method)
    
    curves_SMA_dict_data[CONDITION] = SMZ_evoked_SMR_left
    ci_epochs_SMA_data[CONDITION] = SMZ_epochs_SMR

    fill_1 = filler_between(axes, ylims)

    set_axis_properties(axes,
                        ylims=ylims, 
                        tlims=(tmin, tmax), 
                        title=f'HbO response in SMA in LEFT hemisphere\n {SUBJECT} in {subj_n} subjects, Condition {CONDITION}',
                        xlabel='Time, s', 
                        ylabel='Hb concentration, Δ μM\L',  
                        linewidth=2, 
                        fontsize=16, 
                        title_size=18,
                        legend_flag=False
                            )
    ### LEGENDS PARAMS ###
    ax_legend = fig.add_axes([0.75, 0.10, 0.25, 0.25])
    ax_legend.legend([line_smz_smr, 
                      fill_1], 
                     [f'SMA {curves_hb} ', 
                     'Task duration'], 
                    **legend_params_dict) 

    ax_legend.set_frame_on(False)
    ax_legend.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
    curves_SMA_dict_data[CONDITION] = fig
    
    plt.subplots_adjust(**curves_subplot_params)
    fig.savefig(f'/home/avaldamon/Desktop/mean_pictures/delta/HAEMO_{CONDITION}_SMA.png')
    plt.close()
    fig.clear()

    topo_m1 = data[:, m1_group_left]
    topo_s1 = data[:, s1_group_left]
    topo_smz = data[:, smz_group_left]
    
    top_3_indices_M1 = np.argpartition(topo_m1, -3, axis=1)[:, -3:]
    top_indices_M1 = np.zeros_like(top_3_indices_M1)
    top_3_indices_S1 = np.argpartition(topo_s1, -3, axis=1)[:, -3:]
    top_indices_S1 = np.zeros_like(top_3_indices_S1)

    for i, row in enumerate(top_3_indices_M1):
        top_indices_M1[i] = np.take(m1_group_left, row)
    for i, row in enumerate(top_3_indices_S1):
        top_indices_S1[i] = np.take(s1_group_left, row)
    
    top_chans_M1_dict[CONDITION] = top_indices_M1
    top_chans_S1_dict[CONDITION] = top_indices_S1

    
    topo_m1 = np.mean(np.partition(topo_m1, -3, axis=1)[:, -3:], axis=1)
    topo_s1 = np.mean(np.partition(topo_s1, -3, axis=1)[:, -3:], axis=1)
    topo_smz = np.mean(np.partition(topo_smz, -3, axis=1)[:, -3:], axis=1)
    
    print(pool_m1)
    
#     topo_m1 = np.median(data[:, m1_group_left], axis=1)
#     topo_s1 = np.median(data[:, s1_group_left], axis=1)
#     topo_smz = np.median(data[:, smz_group_left], axis=1)

    
    df_initial[f'{CONDITION} M1'] = topo_m1
#     df[f'{CONDITION}_M1_right'] = M1_right_median
    
    df_initial[f'{CONDITION} S1'] = topo_s1
#     df[f'{CONDITION}_S1_right'] = S1_right_median

    df_smz[f'{CONDITION}_SMA_left'] = topo_smz

clim = dict(kind="value", lims=[3, 4.5, 6])
for i in range(4):
    ind=i
    CONDITION = CONDITION_LIST[ind]
    stc = stc_list[ind]

    surfer_kwargs = dict(
                        subject='fsaverage',
                        surface='pial',
                        time_label=None,
                        hemi="both",
                        subjects_dir=subjects_dir,
                        views="axial",
                        initial_time=None,
                        time_unit="s",
                        clim=clim,
                        size=(1200, 800),
                        smoothing_steps=10,
                        background='w',
                        colorbar=False,
                        time_viewer=False,
                        show_traces=False
    )

    # Plot surface
    brain = stc.plot(**surfer_kwargs, 
                     colormap='magma'
                    )
    labels = (
        "ctx-lh-precentral",
        "ctx-lh-postcentral",
    )

    peak_vertex, peak_time = stc.get_peak(hemi="lh", vert_as_index=True, time_as_index=True)
    peak_vertex_surf = stc.lh_vertno[peak_vertex]
    peak_value = stc.lh_data[peak_vertex, peak_time]

    # brain.add_sensors(info=info_hbo_total, trans='fsaverage', fnirs=dict(channels=0.8))
    #     brain.add_sensors(info=eeg_info, trans='fsaverage', eeg='original' )
    brain.add_sensors(info=m1_info, trans='fsaverage', fnirs=dict(channels=0.75),
                     sensor_colors=desatured_red)
    brain.add_sensors(info=s1_info, trans='fsaverage', fnirs=dict(channels=0.75),
                     sensor_colors=desatured_blue)
    brain.add_sensors(info=between_info, trans='fsaverage', fnirs=dict(channels=0.75),
                     sensor_colors='#007C59')
    # brain.add_volume_labels(aseg="aparc+aseg", labels=labels, colors=[blue, red])
    # brain.add_foci(peak_vertex_surf, coords_as_verts=True, hemi="lh", color="green")

    mne.viz.set_3d_view(figure=brain, 
                        azimuth=170.0,
                        elevation=45,
                        focalpoint='auto',
                        distance=500
                       )
    source_spaces_dict_data[CONDITION] = brain
    brain.add_text(0.1, 0.9, f"{CONDITION}", "title", font_size=16)
    screenshot = brain.screenshot()
    nonwhite_pix = (screenshot != 255).any(-1)
    nonwhite_row = nonwhite_pix.any(1)
    nonwhite_col = nonwhite_pix.any(0)
    cropped_screenshot = screenshot[nonwhite_row][:, nonwhite_col]
    screenshots_dict[CONDITION] = cropped_screenshot
    brain.save_image(f'/home/avaldamon/Desktop/mean_pictures/delta/INVERSE_{CONDITION}.png')
    brain.close()


Permuting 16383 times (exact test)...
    Reading a source space...
    [done]
    Reading a source space...
    [done]
    2 source spaces read


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:    0.0s finished


[8.81469727e-01 6.51062012e-01 2.18811035e-01 4.14428711e-02
 1.93908691e-01 1.76086426e-01 4.02526855e-01 1.32263184e-01
 3.12072754e-01 1.61376953e-01 6.24877930e-01 8.10546875e-02
 4.94384766e-03 8.99658203e-02 8.38012695e-02 6.80541992e-02
 9.85351562e-01 1.25122070e-02 2.31323242e-02 4.57763672e-03
 2.12402344e-02 5.14770508e-01 2.68554688e-02 1.00000000e+00
 7.84545898e-01 7.27661133e-01 2.44140625e-04 1.22070312e-04
 8.54492188e-04 6.10351562e-05 1.22070312e-04 4.33349609e-03
 6.10351562e-05 2.44140625e-04 6.10351562e-05 6.71386719e-04
 7.93457031e-04 5.49316406e-04 1.22070312e-04 8.91113281e-03
 1.38000488e-01 5.49316406e-04 1.57470703e-02 8.54492188e-04
 7.65991211e-02 4.57763672e-03 7.73315430e-01 1.95922852e-02
 1.00708008e-02 9.19555664e-01 4.13208008e-02 6.10351562e-05
 1.64794922e-03 5.49316406e-04 6.10351562e-05 3.05175781e-04
 8.54492188e-04 3.05175781e-04 6.10351562e-05 1.14746094e-02
 6.59179688e-03 9.99389648e-01 6.10351562e-05 5.55419922e-03
 3.07983398e-01 4.851074

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:    0.0s finished


[7.76611328e-01 6.37207031e-01 9.54589844e-01 9.97558594e-01
 1.00000000e+00 9.99877930e-01 1.00000000e+00 9.33654785e-01
 2.78747559e-01 1.12060547e-01 1.00000000e+00 4.94384766e-03
 3.22692871e-01 9.59350586e-01 9.70520020e-01 6.78100586e-01
 1.00000000e+00 9.90173340e-01 1.00000000e+00 9.94445801e-01
 9.52270508e-01 9.91760254e-01 8.00598145e-01 9.99816895e-01
 9.94506836e-01 1.90429688e-01 1.84936523e-02 2.42858887e-01
 1.89208984e-03 2.00378418e-01 1.28173828e-03 7.47070312e-02
 5.52551270e-01 2.29553223e-01 4.39514160e-01 5.97351074e-01
 7.57019043e-01 4.37133789e-01 4.44091797e-01 4.77294922e-02
 3.98132324e-01 4.96215820e-02 1.20849609e-02 7.70263672e-02
 6.34765625e-03 3.47900391e-03 4.95666504e-01 2.01416016e-03
 9.76562500e-04 3.66210938e-04 2.76000977e-01 2.27722168e-01
 7.81250000e-03 4.02832031e-03 6.04248047e-03 8.62365723e-01
 1.00000000e+00 9.91577148e-01 9.35058594e-01 9.21813965e-01
 9.94323730e-01 4.08935547e-03 2.27722168e-01 2.84729004e-01
 3.68286133e-01 1.000000

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:    0.0s finished


[5.00488281e-03 2.63671875e-02 1.22192383e-01 8.97216797e-03
 2.78869629e-01 1.39343262e-01 4.55932617e-02 1.50451660e-01
 7.41821289e-01 5.98754883e-02 3.72314453e-03 3.05175781e-04
 3.16772461e-02 1.10473633e-02 8.17871094e-03 2.56347656e-03
 3.72436523e-01 4.51660156e-03 2.63366699e-01 8.47778320e-02
 1.15112305e-01 2.17285156e-01 4.29687500e-02 8.82263184e-01
 5.82702637e-01 4.32800293e-01 1.22070312e-04 1.22070312e-04
 1.15966797e-03 8.54492188e-04 1.22070312e-04 1.37939453e-02
 1.22070312e-04 4.88281250e-04 2.92968750e-03 4.88281250e-04
 3.41796875e-03 2.05078125e-02 3.96728516e-03 1.58081055e-02
 5.12695312e-03 1.83105469e-03 8.17871094e-03 1.47094727e-02
 1.68457031e-02 1.22070312e-04 5.23071289e-02 4.72412109e-02
 2.66723633e-02 5.40161133e-02 1.83105469e-03 1.22070312e-04
 9.88769531e-03 1.15966797e-03 1.22070312e-04 4.94384766e-03
 4.69970703e-02 5.31005859e-03 3.48510742e-02 3.78417969e-02
 1.34277344e-03 1.17431641e-01 1.39770508e-02 3.10058594e-02
 1.44348145e-01 1.274414

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:    0.0s finished


[2.09350586e-02 9.86328125e-02 5.42358398e-01 3.23425293e-01
 4.71801758e-01 3.88427734e-01 8.58642578e-01 5.04150391e-01
 3.86169434e-01 1.90246582e-01 2.28149414e-01 6.40869141e-03
 3.40148926e-01 2.05688477e-01 5.09826660e-01 1.31347656e-01
 1.00000000e+00 5.37109375e-03 4.98046875e-01 5.45349121e-01
 4.50073242e-01 5.88989258e-01 5.04455566e-01 1.00000000e+00
 5.90576172e-01 1.00000000e+00 5.43212891e-03 3.66210938e-04
 3.72924805e-02 1.83105469e-04 3.66210938e-04 4.20532227e-02
 1.22070312e-04 3.29589844e-03 1.86767578e-02 2.32543945e-02
 1.03759766e-02 1.92871094e-02 2.07519531e-03 6.65283203e-03
 3.05786133e-02 5.43212891e-03 1.00000000e+00 2.40234375e-01
 1.28173828e-03 6.10351562e-05 1.00000000e+00 6.53076172e-03
 1.09863281e-03 1.83105469e-04 3.05175781e-04 4.51660156e-03
 6.10351562e-05 1.22070312e-04 6.10351562e-05 7.56835938e-03
 1.93542480e-01 5.20019531e-02 3.04931641e-01 2.75207520e-01
 3.09936523e-01 2.49023438e-02 5.55419922e-03 1.70898438e-03
 2.41271973e-01 6.786499

In [265]:
def new_top_chans_indices(dicr, roi='M1'):
    keys = dicr.keys()
    hh_dict = dict()
    for i in keys:
        flat_arr = dicr[i].flatten()
        unique_items, counts = np.unique(flat_arr, return_counts=True)
        if roi=='M1':    
            names = [ 6,  7,  8,  9, 10, 11]
        else:
            names = [0, 1, 2, 3, 4, 5]
            
        ranks = [(names[i], counts[i]) for i in range(len(names))]
        ranks = sorted(ranks, key=lambda x: x[1], reverse=True)
        hh_dict[i] = ranks[0:3]
    return hh_dict

top_chs_M1 = new_top_chans_indices(top_chans_M1_dict, 'M1')
top_chs_S1 = new_top_chans_indices(top_chans_S1_dict, 'S1')

In [186]:
# topo_dict_data 
# sign_sensors_data

# curves_M1_dict_data
# ci_epochs_M1_data

# curves_S1_dict_data
# ci_epochs_S1_data

# curves_SMA_dict_data
# ci_epochs_SMA_data

# source_spaces_dict_data


In [223]:
from mpl_toolkits.axes_grid1 import ImageGrid, inset_locator, make_axes_locatable

fig, axes = plt.subplots(2, 2, figsize=(20, 18))
counter = 0
CONDITIONS = ['ME', 'TS', 'MI', 'TI']

limits = [-2, 17]


cm_colors_topo = [(0, fnirs_colors['hbo']), (0.35,   '#5D8B35'),
(0.5, 'black'), (0.6, 'white'),  (0.75, 'white'), 
                 (1.0, fnirs_colors['hbr'])]
cm_topo_curves_cmap = LinearSegmentedColormap.from_list('Custom_cmap', cm_colors_topo, )

for i in range(2):
    for j in range(2):

        line_m1, = axes[i][j].plot(times, curves_M1_dict_data[CONDITIONS[counter]], 
                                       label=f'M1/{curves_hb} SMR', 
             color=fnirs_colors['hbo'])
        make_ci(ax=axes[i][j], to_plot_around=curves_M1_dict_data[CONDITIONS[counter]], 
            epochs_data=ci_epochs_M1_data[CONDITIONS[counter]], 
            alpha=0.25,  color=fnirs_colors['hbo'], multiply=False, n_boot=n_boot,
           func=func, method=method)
        
        line_s1, = axes[i][j].plot(times, curves_S1_dict_data[CONDITIONS[counter]], 
                                       label=f'S1/{curves_hb} SMR', 
             color=fnirs_colors['hbr'])
        make_ci(ax=axes[i][j], to_plot_around=curves_S1_dict_data[CONDITIONS[counter]], 
            epochs_data=ci_epochs_S1_data[CONDITIONS[counter]], 
            alpha=0.25,  color=fnirs_colors['hbr'], multiply=False, n_boot=n_boot,
           func=func, method=method)
        fig.subplots_adjust(top=0.935,
                            bottom=0.175,
                            left=0.09,
                            right=0.985,
                            hspace=0.25,
                            wspace=0.07)
        
        fill_1 = filler_between(axes[i][j], limits)
        
        if i==1:
            set_axis_properties(axes[i][j],
                            ylims=limits, 
                            tlims=(tmin, tmax), 
                            title=f'{CONDITIONS[counter]}',
                            xlabel='Time, s', 
                            ylabel='',  
                            linewidth=2, 
                            fontsize=16, 
                            title_size=18,
                            legend_flag=False
                                )
        else:
            set_axis_properties(axes[i][j],
                            ylims=limits, 
                            tlims=(tmin, tmax), 
                            title=f'{CONDITIONS[counter]}',
                            xlabel='', 
                            ylabel='',  
                            linewidth=2, 
                            fontsize=16, 
                            title_size=18,
                            legend_flag=False
                                )
        wh = '38%'
        axins = inset_locator.inset_axes(axes[i][j], width=wh, 
                                         height=wh, loc='upper right')
        
        topo_linewidth = 0.5
        
        indices_M1 = [i[0] for i in top_chs_M1[CONDITIONS[counter]]]
        indices_S1 = [i[0] for i in top_chs_S1[CONDITIONS[counter]]]

        left_m1_roi = get_channel_indices(M1_LEFT_ROI_HBO,
                        info=info_left_smz)
        left_m1_roi = [x for x in left_m1_roi if x not in indices_M1]
        left_s1_roi = get_channel_indices(S1_LEFT_ROI_HBO, 
                        info=info_left_smz)
        left_s1_roi = [x for x in left_s1_roi if x not in indices_S1]
        left_remaining_roi = get_channel_indices(REMAINING_LEFT_ROI, 
                        info=info_left_smz)
        
        a = mne.viz.plot_sensors(
info_left_smz, 
ch_groups=[
#red       
    left_m1_roi, 
#grey
     indices_M1+indices_S1,  
#black
   left_remaining_roi,  
#blue
      left_s1_roi, 


],
sphere=(0.0, 0.0185, 0.0, 0.085),

axes=axins,
pointsize=50, 
linewidth=topo_linewidth,
            cmap=cm_topo_curves_cmap,
        )
#         target_chans = ([-0.04930527091026306, 0.023039783409330994],
#                         [0.04507893744856119, 0.02398763585370034],
#                         [-0.0012414913401007652, 0.023633225545287132])
#         for coord in target_chans:
#             axins.plot(*coord, marker='o', color='#717171', ms=5)
        
        counter += 1

        
ax_legend = fig.add_axes([0.55, 0.075, 0.0, 0.0])

legend_params_dict = dict(
                    loc='center', 
                    fontsize=14,       
                    borderpad=1.5, 
                    labelspacing=1.,
                    markerscale=10, 
                    framealpha=1
)       
ax_legend.legend([line_m1,
                  line_s1,
              fill_1], 
             ['M1 ROI', 
             'S1 ROI',
             'Task duration'], ncols=3,
            **legend_params_dict)
ax_legend.set_frame_on(False)
ax_legend.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)


left_ax = fig.add_axes([0.035, 0.175, 0.0, 0.8])
left_ax.text(0.5, 0.5,'HbO2 concentration, Δ μM\L', 
#                      fontproperties=font,
             fontsize=16,
             rotation='vertical', va='center', ha='center', 
#                      transform=left_ax.transAxes
            )
left_ax.axis('off')
        
        
fig.savefig('/home/avaldamon/Desktop/mean_pictures/CURVES_M1_S1.png', dpi=300)


        

In [55]:
fig, axes = plt.subplots(2, 2, figsize=(9, 8))
counter = 0
CONDITIONS = ['ME', 'TS', 'MI', 'TI']

ylims=[-5, 5]
# topo_dict_data 
# sign_sensors_data
mask_params = dict(marker='o', 
                   markerfacecolor='black', 
                   markeredgecolor='white',
                   linewidth=0, 
                   markersize=8
                   )

for i in range(2):
    for j in range(2):
        a = mne.viz.plot_topomap(
                                data=topo_dict_data[CONDITIONS[counter]],
                                pos=info_hbo_total,
                                vlim=ylims,
                                axes=axes[i][j],
                                contours=3,
                                extrapolate='local',
                                image_interp='linear',
    #                             cmap=custom_cmap,
                                cmap='seismic',
                                sphere=(0.0, 0.0185, 0.0, 0.085),
    #                             mask=rel_mask_SMR,
                                mask = sign_sensors_data[CONDITIONS[counter]],
                                mask_params=mask_params,
                                show=False,
    #                             names=info_hbo_total.ch_names
        )

        target_chans = ([-0.04930527091026306, 0.023039783409330994],
                        [0.04507893744856119, 0.02398763585370034],
                        [-0.0012414913401007652, 0.023633225545287132])
        for coord in target_chans:
            axes[i][j].plot(*coord, marker='o', color='#717171', ms=9)

        ### TITLES ###
        axes[i][j].set_title(
                f'{CONDITIONS[counter]}', loc='center',
        fontsize=20)
        
        
        sm = plt.cm.ScalarMappable(
#                         cmap=custom_cmap,
                        cmap='seismic',
                        norm=matplotlib.colors.Normalize(vmin=ylims[0], 
                                                         vmax=ylims[1]))

        cbaxes = fig.add_axes([0.075, 0.25, 0.04, 0.5]) # setup colorbar axes. 
        cbar = plt.colorbar(mappable=sm, cax=cbaxes, 
                            pad=0.4, orientation='vertical')
        cbar.set_label('HbO2 concentration, Δ μM\L', loc='center', size=14)
        cbar.ax.yaxis.set_label_coords(-0.65, 0.5)
        cbar.ax.tick_params(labelsize=12)

        fig.subplots_adjust(top=0.955,
                            bottom=0.0,
                            left=0.145,
                            right=1.0,
                            hspace=0.005,
                            wspace=0.0)
    
        

        counter += 1
fig.savefig('/home/avaldamon/Desktop/mean_pictures/TOPOMAP.png', dpi=300)


In [54]:
clim = dict(kind="value", lims=[3, 4.5, 6])

fig, axes = plt.subplots(2, 2, figsize=(10, 10))
counter = 0
CONDITIONS = ['ME', 'TS', 'MI', 'TI']

for i in range(2):
    for j in range(2):
        condition = CONDITIONS[counter]
        axes[i][j].imshow(screenshots_dict[condition])
        axes[i][j].axis('off')
        axes[i][j].set_title(condition, size=16)
        counter += 1

fig.subplots_adjust(
                    top=0.93,
                    bottom=0.145,
                    left=0.025,
                    right=0.975,
                    hspace=0.215,
                    wspace=0.105
)
cbaxes = fig.add_axes([0.25, 0.05, 0.5, 0.04]) # setup colorbar axes. 
cbar = mne.viz.plot_brain_colorbar(cbaxes, 
                                   clim, 
                                   colormap='magma', 
                                   label='HbO2 concentration, Δ μM\L',
                                  orientation='horizontal')



fig.savefig('/home/avaldamon/Desktop/mean_pictures/BRAIN.png', dpi=300)


In [301]:
smz_dataframe = df_smz.copy()

smz_stats_dict = OrderedDict(smz_ME_MI = pg.wilcoxon(smz_dataframe['ME_SMA_left'], smz_dataframe['MI_SMA_left']),
smz_ME_TS = pg.wilcoxon(smz_dataframe['ME_SMA_left'], smz_dataframe['TS_SMA_left']),
smz_MI_TI = pg.wilcoxon(smz_dataframe['MI_SMA_left'], smz_dataframe['TI_SMA_left']),
smz_TS_TI = pg.wilcoxon(smz_dataframe['TS_SMA_left'], smz_dataframe['TI_SMA_left']))

chosen_pval = 0.05/4

items = list(smz_stats_dict.values())
keys = list(smz_stats_dict.keys())
stats_df = pd.concat(items)
stats_df['Comparison'] = keys
stats_df['Significance'] = stats_df['p-val'] < chosen_pval

In [302]:
stats_df

,W-val,alternative,p-val,RBC,CLES,Comparison,Significance
Wilcoxon,25.0,two-sided,0.047913,0.583333,0.702222,smz_ME_MI,False
Wilcoxon,23.0,two-sided,0.035339,0.616667,0.764444,smz_ME_TS,False
Wilcoxon,33.0,two-sided,0.135376,0.450000,0.582222,smz_MI_TI,False
Wilcoxon,57.0,two-sided,0.890381,0.050000,0.462222,smz_TS_TI,False


In [303]:
smz_dict_cohen = OrderedDict(ME_MI_effect_size = pg.compute_effsize(smz_dataframe['ME_SMA_left'], smz_dataframe['MI_SMA_left'], 
                                       paired=True, eftype='cohen'),
ME_TS_effect_size = pg.compute_effsize(smz_dataframe['ME_SMA_left'], smz_dataframe['TS_SMA_left'], 
                                       paired=True, eftype='cohen'),
MI_TI_effect_size = pg.compute_effsize(smz_dataframe['MI_SMA_left'], smz_dataframe['TI_SMA_left'], 
                                       paired=True, eftype='cohen'),
TS_TI_effect_size = pg.compute_effsize(smz_dataframe['TS_SMA_left'], smz_dataframe['TI_SMA_left'], 
                                       paired=True, eftype='cohen'))

smz_dict_cohen.items()

odict_items([('ME_MI_effect_size', 0.5690435426643732), ('ME_TS_effect_size', 0.9676890895218809), ('MI_TI_effect_size', 0.41867284347424677), ('TS_TI_effect_size', 0.00608835104146385)])

In [304]:
results_dict = {}
for i, col1 in enumerate(smz_dataframe.columns):
    for j, col2 in enumerate(smz_dataframe.columns):
        if i < j:
            ttest_result = pg.ttest(smz_dataframe[col1], smz_dataframe[col2], paired=True)
            results_dict[(col1, col2)] = (ttest_result['T'].values[0], ttest_result['p-val'].values[0])

results_df = pd.DataFrame.from_dict(results_dict, orient='index', columns=['T-statistic', 'p-value'])

num_comparisons = len(results_df)
bonferroni_alpha = 0.05 / 6

results_df['significance'] = results_df['p-value'] < bonferroni_alpha
results_df['significance'] = results_df['significance'].apply(lambda x: 'Yes' if x else 'No')

print(results_df)

                            T-statistic   p-value significance
(ME_SMA_left, TS_SMA_left)     2.807169  0.013982           No
(ME_SMA_left, MI_SMA_left)     1.437561  0.172533           No
(ME_SMA_left, TI_SMA_left)     2.625166  0.019974           No
(TS_SMA_left, MI_SMA_left)    -0.929124  0.368569           No
(TS_SMA_left, TI_SMA_left)     0.014580  0.988573           No
(MI_SMA_left, TI_SMA_left)     1.450207  0.169034           No


In [305]:
dataframe = df_initial.copy()
# dataframe.index = ids_of_subjs

In [306]:
column = dataframe.melt().to_numpy()[:, 1]
cond_column = []
for i in CONDITION_LIST:
    cond_column.extend([i]*new_shape[0]*2)
df = pd.DataFrame({
'subj': [i for i in range(new_shape[0])]*len(CONDITION_LIST)*2,
'condition': cond_column,
'imagery': ['Imagery' if 'I' in i else 'Real' for i in cond_column],
'activity':['Motor' if i[0]=="M" else 'Tactile' for i in cond_column],
'roi': (['M1']*new_shape[0] + ['S1']*new_shape[0])*len(CONDITION_LIST),
'values': [float(i) for i in column]}
)

In [307]:
from scipy.stats import shapiro

# pg.normality(dataframe)

imagery_levels = df['imagery'].unique()
activity_levels = df['activity'].unique()
roi_levels = df['roi'].unique()

for imagery_level in imagery_levels:
    for activity_level in activity_levels:
        for roi_level in roi_levels:
            subset = df[(df['imagery'] == imagery_level) & (df['activity'] == activity_level) & (df['roi'] == roi_level)]
            normality_test_statistic, p_value = shapiro(subset['values'])
            print(f"Imagery: {imagery_level}, Activity: {activity_level}, ROI: {roi_level}")
            print(f"Normality Test Statistic: {normality_test_statistic}, p-value: {p_value}")


Imagery: Real, Activity: Motor, ROI: M1
Normality Test Statistic: 0.964499831199646, p-value: 0.7699505686759949
Imagery: Real, Activity: Motor, ROI: S1
Normality Test Statistic: 0.9407981634140015, p-value: 0.39251312613487244
Imagery: Real, Activity: Tactile, ROI: M1
Normality Test Statistic: 0.9402036666870117, p-value: 0.38493821024894714
Imagery: Real, Activity: Tactile, ROI: S1
Normality Test Statistic: 0.9561707377433777, p-value: 0.6262142062187195
Imagery: Imagery, Activity: Motor, ROI: M1
Normality Test Statistic: 0.8191072940826416, p-value: 0.0065351855009794235
Imagery: Imagery, Activity: Motor, ROI: S1
Normality Test Statistic: 0.8787344694137573, p-value: 0.0454285591840744
Imagery: Imagery, Activity: Tactile, ROI: M1
Normality Test Statistic: 0.8556176424026489, p-value: 0.02087615802884102
Imagery: Imagery, Activity: Tactile, ROI: S1
Normality Test Statistic: 0.969682514667511, p-value: 0.8533252477645874


In [308]:
pg.anova(data=df, dv='values', between=['imagery', 'roi', 'activity'])

,Source,SS,DF,MS,F,p-unc,np2
0,imagery,4.196899,1.0,4.196899,0.487164,0.486641,0.004331
1,roi,20.784275,1.0,20.784275,2.412581,0.123185,0.021087
2,activity,133.520535,1.0,133.520535,15.498694,0.000144,0.121560
3,imagery * roi,0.127157,1.0,0.127157,0.014760,0.903520,0.000132
4,imagery * activity,19.949023,1.0,19.949023,2.315627,0.130898,0.020256
5,roi * activity,124.743809,1.0,124.743809,14.479916,0.000231,0.114484
6,imagery * roi * activity,13.304981,1.0,13.304981,1.544405,0.216559,0.013602
7,Residual,964.874862,112.0,8.614954,NaN,NaN,NaN


In [309]:
df_motor = df[df['condition'].str.contains('M')]
df_tactile = df[df['condition'].str.contains('T')]
df_imagery = df[df['condition'].str.contains('I')]
df_real = df[~df['condition'].str.contains('I')]
df_m1 = df[df['roi'].str.contains('M')]
df_s1 = df[df['roi'].str.contains('S')]

In [310]:
pg.rm_anova(data=df_m1, dv='values', within=['imagery', 'activity'], subject='subj')

,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,ng2,eps
0,imagery,2.892552,1,14,2.892552,0.183728,0.674710,0.674710,0.005414,1.0
1,activity,258.189757,1,14,258.189757,36.390619,0.000031,0.000031,0.327018,1.0
2,imagery * activity,32.918759,1,14,32.918759,4.115624,0.061953,0.061953,0.058340,1.0


In [311]:
pg.rm_anova(data=df_s1, dv='values', within=['imagery', 'activity'], subject='subj')

,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,ng2,eps
0,imagery,1.431504,1,14,1.431504,0.136983,0.716838,0.716838,0.003291,1.0
1,activity,0.074588,1,14,0.074588,0.010881,0.918403,0.918403,0.000172,1.0
2,imagery * activity,0.335244,1,14,0.335244,0.056373,0.815763,0.815763,0.000773,1.0


In [312]:

# pg.rm_anova(data=df, dv='values', within=['imagery', 'roi'], subject='subj')

# pg.rm_anova(data=df, dv='values', within=['imagery', 'condition'], subject='subj')

# pg.rm_anova(data=df, dv='values', within=['imagery', 'activity'], subject='subj')

pg.rm_anova(data=df_motor, dv='values', within=['imagery', 'roi'], subject='subj')

,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,ng2,eps
0,imagery,21.223045,1,14,21.223045,1.190851,0.293583,0.293583,0.038876,1.0
1,roi,21.845385,1,14,21.845385,4.295407,0.057159,0.057159,0.039971,1.0
2,imagery * roi,8.016769,1,14,8.016769,2.238213,0.156836,0.156836,0.015049,1.0


In [313]:
pg.rm_anova(data=df_tactile, dv='values', within=['imagery', 'roi'], subject='subj')

,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,ng2,eps
0,imagery,2.922877,1,14,2.922877,0.175944,0.681248,0.681248,0.006596,1.0
1,roi,123.682700,1,14,123.682700,37.632093,0.000026,0.000026,0.219348,1.0
2,imagery * roi,5.415369,1,14,5.415369,2.550630,0.132569,0.132569,0.012153,1.0


In [314]:
pg.rm_anova(data=df_imagery, dv='values', within=['activity', 'roi'], subject='subj')

,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,ng2,eps
0,activity,25.124662,1,14,25.124662,3.988280,0.065638,0.065638,0.057643,1.0
1,roi,12.081405,1,14,12.081405,3.329290,0.089466,0.089466,0.028573,1.0
2,activity * roi,28.284808,1,14,28.284808,10.823812,0.005369,0.005369,0.064426,1.0


In [315]:
pg.rm_anova(data=df_real, dv='values', within=['activity', 'roi'], subject='subj')

,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,ng2,eps
0,activity,128.344896,1,14,128.344896,8.507743,0.011262,0.011262,0.188057,1.0
1,roi,8.830027,1,14,8.830027,2.242061,0.156505,0.156505,0.015685,1.0
2,activity * roi,109.763982,1,14,109.763982,28.166816,0.000111,0.000111,0.165333,1.0


In [316]:
ttest_kwargs =  dict(dv='values',
                     parametric='False',
                     subject='subj', 
#                      alpha=0.05, 
#                      padjust='fdr_bh',
                     effsize='cohen',
#                      interaction=True
                    )
# pg.wilcoxon

In [317]:
def wilcoxon(bigger_cond, lesser_cond, alternative='greater'):
    return pg.wilcoxon(
        x=dataframe[bigger_cond], 
        y=dataframe[lesser_cond], 
#         paired=True, 
        alternative=alternative
)

def cohen(cond1, cond2):
    x=dataframe[cond1], 
    y=dataframe[cond2], 
    return pg.compute_effsize(x, y, paired=True, eftype='cohen')


In [318]:
statistics = OrderedDict(
ME_M1_ME_S1 = wilcoxon('ME M1', 'ME S1', 'two-sided'),
MI_M1_MI_S1 = wilcoxon('MI M1', 'MI S1', 'two-sided'),
TS_M1_TS_S1 = wilcoxon('TS M1', 'TS S1', 'two-sided'),
TI_M1_TI_S1 = wilcoxon('TI M1', 'TI S1', 'two-sided'),

ME_M1_MI_M1 = wilcoxon('ME M1', 'MI M1', 'two-sided'),
ME_S1_MI_S1 = wilcoxon('ME S1', 'MI S1', 'two-sided'),

TS_M1_TI_M1 = wilcoxon('TS M1', 'TI M1', 'two-sided'),
TS_S1_TI_S1 = wilcoxon('TS S1', 'TI S1', 'two-sided'),

MI_M1_TI_M1 = wilcoxon('MI M1', 'TI M1', 'two-sided'),
TI_S1_MI_S1 = wilcoxon('TI S1', 'MI S1', 'two-sided'),

ME_M1_TS_M1 = wilcoxon('ME M1', 'TS M1', 'two-sided'),
ME_S1_TS_S1 = wilcoxon('ME S1', 'TS S1', 'two-sided'), )


chosen_pval = 0.05/12
items = list(statistics.values())
keys = list(statistics.keys())
stats_df = pd.concat(items)
stats_df['Comparison'] = keys
stats_df['Significance'] = stats_df['p-val'] < chosen_pval




In [319]:
stats_df

,W-val,alternative,p-val,RBC,CLES,Comparison,Significance
Wilcoxon,23.0,two-sided,0.035339,0.616667,0.671111,ME_M1_ME_S1,False
Wilcoxon,50.0,two-sided,0.599487,0.166667,0.533333,MI_M1_MI_S1,False
Wilcoxon,1.0,two-sided,0.000122,-0.983333,0.213333,TS_M1_TS_S1,True
Wilcoxon,4.0,two-sided,0.000427,-0.933333,0.235556,TI_M1_TI_S1,True
Wilcoxon,24.0,two-sided,0.041260,0.600000,0.697778,ME_M1_MI_M1,False
Wilcoxon,54.0,two-sided,0.761536,0.100000,0.568889,ME_S1_MI_S1,False
Wilcoxon,44.0,two-sided,0.389404,-0.266667,0.346667,TS_M1_TI_M1,False
Wilcoxon,57.0,two-sided,0.890381,0.050000,0.484444,TS_S1_TI_S1,False
Wilcoxon,10.0,two-sided,0.002625,0.833333,0.782222,MI_M1_TI_M1,True
Wilcoxon,50.0,two-sided,0.599487,0.166667,0.560000,TI_S1_MI_S1,False


In [320]:
cohens = OrderedDict(
ME_M1_ME_S1 = cohen('ME M1', 'ME S1'),
MI_M1_MI_S1 = cohen('MI M1', 'MI S1'),
TS_M1_TS_S1 = cohen('TS M1', 'TS S1'),
TI_M1_TI_S1 = cohen('TI M1', 'TI S1'),

ME_M1_MI_M1 = cohen('ME M1', 'MI M1'),
ME_S1_MI_S1 = cohen('ME S1', 'MI S1'),

TS_M1_TI_M1 = cohen('TS M1', 'TI M1'),
TS_S1_TI_S1 = cohen('TS S1', 'TI S1'),

MI_M1_TI_M1 = cohen('MI M1', 'TI M1'),
TI_S1_MI_S1 = cohen('TI S1', 'MI S1'),

ME_M1_TS_M1 = cohen('ME M1', 'TS M1'),
ME_S1_TS_S1 = cohen('ME S1', 'TS S1'), )

cohen_df = pd.DataFrame(data=list(cohens.values()), index=list(cohens.keys()))

In [321]:
cohen_df

,0
ME_M1_ME_S1,0.644634
MI_M1_MI_S1,0.152733
TS_M1_TS_S1,-1.058882
TI_M1_TI_S1,-1.018806
ME_M1_MI_M1,0.577488
ME_S1_MI_S1,0.165432
TS_M1_TI_M1,-0.370448
TS_S1_TI_S1,0.057066
MI_M1_TI_M1,0.925645
TI_S1_MI_S1,0.031305


In [322]:
pg.wilcoxon(
            x=dataframe['ME M1'], 
            y=dataframe['ME S1'], 
#             paired=True, 
            alternative='greater')

pg.wilcoxon(
            x=dataframe['MI M1'], 
            y=dataframe['MI S1'], 
#             paired=True, 
            alternative='greater')

pg.wilcoxon(
        x=dataframe['TI M1'], 
        y=dataframe['TI S1'], 
#         paired=True, 
        alternative='less'
)

pg.wilcoxon(
        x=dataframe['TS M1'], 
        y=dataframe['TS S1'],
#         paired=True, 
        alternative='less')

,W-val,alternative,p-val,RBC,CLES
Wilcoxon,1.0,less,0.000061,-0.983333,0.786667


In [323]:
ttest_kwargs =  dict(dv='values',
                     parametric='False',
                     subject='subj', 
#                      alpha=0.05, 
#                      padjust='fdr_bh',
                     effsize='cohen',
#                      interaction=True
                    )
# pg.wilcoxon

In [324]:
pg.pairwise_tests(data=df_motor, 
                  within=['roi', 'imagery'],
                  **ttest_kwargs)

,Contrast,roi,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,cohen
0,roi,-,M1,S1,True,False,2.072536,14.0,two-sided,0.057159,1.387,0.601999
1,imagery,-,Imagery,Real,True,False,-1.091261,14.0,two-sided,0.293583,0.436,-0.443214
2,roi * imagery,M1,Imagery,Real,True,False,-1.353064,14.0,two-sided,0.197482,0.563,-0.577488
3,roi * imagery,S1,Imagery,Real,True,False,-0.500449,14.0,two-sided,0.624534,0.293,-0.165432


In [325]:
pg.pairwise_tests(data=df_tactile, 
                  within=['roi', 'imagery'],
                  **ttest_kwargs)



,Contrast,roi,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,cohen
0,roi,-,M1,S1,True,False,-6.134500,14.0,two-sided,0.000026,946.989,-1.611149
1,imagery,-,Imagery,Real,True,False,0.419456,14.0,two-sided,0.681248,0.284,0.173036
2,roi * imagery,M1,Imagery,Real,True,False,0.971549,14.0,two-sided,0.347762,0.394,0.370448
3,roi * imagery,S1,Imagery,Real,True,False,-0.137355,14.0,two-sided,0.892706,0.265,-0.057066


In [326]:
pg.pairwise_tests(data=df_imagery, 
                  within=['roi', 'activity'],
                  **ttest_kwargs)

,Contrast,roi,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,cohen
0,roi,-,M1,S1,True,False,-1.824634,14.0,two-sided,0.089466,0.989,-0.397148
1,activity,-,Motor,Tactile,True,False,1.997068,14.0,two-sided,0.065638,1.248,0.538591
2,roi * activity,M1,Motor,Tactile,True,False,3.792284,14.0,two-sided,0.001981,21.593,0.925645
3,roi * activity,S1,Motor,Tactile,True,False,-0.094833,14.0,two-sided,0.925791,0.263,-0.031305


In [327]:
pg.pairwise_tests(data=df_real, 
                  within=['roi', 'activity'],
                  **ttest_kwargs)

,Contrast,roi,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,cohen
0,roi,-,M1,S1,True,False,-1.497351,14.0,two-sided,0.156505,0.661,-0.338104
1,activity,-,Motor,Tactile,True,False,2.916804,14.0,two-sided,0.011262,5.052,1.038322
2,roi * activity,M1,Motor,Tactile,True,False,4.570144,14.0,two-sided,0.000437,79.113,1.723438
3,roi * activity,S1,Motor,Tactile,True,False,0.218566,14.0,two-sided,0.830142,0.268,0.072862


In [328]:
pg.rm_anova(data=df_motor, dv='values', within=['roi', 'imagery'], subject='subj')

,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,ng2,eps
0,roi,21.845385,1,14,21.845385,4.295407,0.057159,0.057159,0.039971,1.0
1,imagery,21.223045,1,14,21.223045,1.190851,0.293583,0.293583,0.038876,1.0
2,roi * imagery,8.016769,1,14,8.016769,2.238213,0.156836,0.156836,0.015049,1.0


In [329]:
pg.rm_anova(data=df_tactile, dv='values', within=['roi', 'imagery'], subject='subj')

,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,ng2,eps
0,roi,123.682700,1,14,123.682700,37.632093,0.000026,0.000026,0.219348,1.0
1,imagery,2.922877,1,14,2.922877,0.175944,0.681248,0.681248,0.006596,1.0
2,roi * imagery,5.415369,1,14,5.415369,2.550630,0.132569,0.132569,0.012153,1.0


In [330]:
pg.pairwise_tests(data=df_m1, 
                  within=['activity', 'imagery'],
                  **ttest_kwargs)

,Contrast,activity,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,cohen
0,activity,-,Motor,Tactile,True,False,6.032464,14.0,two-sided,0.000031,811.284,2.201141
1,imagery,-,Imagery,Real,True,False,-0.428636,14.0,two-sided,0.674710,0.285,-0.183694
2,activity * imagery,Motor,Imagery,Real,True,False,-1.353064,14.0,two-sided,0.197482,0.563,-0.577488
3,activity * imagery,Tactile,Imagery,Real,True,False,0.971549,14.0,two-sided,0.347762,0.394,0.370448


In [331]:
pg.pairwise_tests(data=df_s1, 
                  within=['activity', 'imagery'],
                  **ttest_kwargs)

,Contrast,activity,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,cohen
0,activity,-,Motor,Tactile,True,False,0.104310,14.0,two-sided,0.918403,0.264,0.036948
1,imagery,-,Imagery,Real,True,False,-0.370113,14.0,two-sided,0.716838,0.279,-0.144965
2,activity * imagery,Motor,Imagery,Real,True,False,-0.500449,14.0,two-sided,0.624534,0.293,-0.165432
3,activity * imagery,Tactile,Imagery,Real,True,False,-0.137355,14.0,two-sided,0.892706,0.265,-0.057066


In [332]:
pg.rm_anova(data=df_real, dv='values', within=['roi', 'activity'], subject='subj')

,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,ng2,eps
0,roi,8.830027,1,14,8.830027,2.242061,0.156505,0.156505,0.015685,1.0
1,activity,128.344896,1,14,128.344896,8.507743,0.011262,0.011262,0.188057,1.0
2,roi * activity,109.763982,1,14,109.763982,28.166816,0.000111,0.000111,0.165333,1.0


In [333]:
pg.rm_anova(data=df_imagery, dv='values', within=['roi', 'activity'], subject='subj')

,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,ng2,eps
0,roi,12.081405,1,14,12.081405,3.329290,0.089466,0.089466,0.028573,1.0
1,activity,25.124662,1,14,25.124662,3.988280,0.065638,0.065638,0.057643,1.0
2,roi * activity,28.284808,1,14,28.284808,10.823812,0.005369,0.005369,0.064426,1.0


In [334]:
colnames = list(dataframe.columns)
colnames.sort(key=lambda x: x[-2:]=='S1')

def swapPositions(list, pos1, pos2):
    temp = list[pos1]
    list[pos1] = list[pos2]
    list[pos2] = temp
    return list

# Example usage
pos1, pos2 = 1, 2
pos3, pos4 = -2, -3# Adjust indices as needed
colnames = swapPositions(colnames, pos1, pos2)
colnames = swapPositions(colnames, pos3, pos4)
colnames_custom = colnames

In [335]:
%matplotlib auto

Using matplotlib backend: QtAgg


In [351]:
dataframe_reordered = dataframe
columns_with_M1 = [col for col in dataframe_reordered.columns if 'M1' in col]

alpha = 0.5
transparent_blue, blue = rgb(desatured_blue, alpha), rgb(desatured_blue, 0.95)
transparent_red, red = rgb(desatured_red, alpha), rgb(desatured_red, 0.95)

palette_transp = {col: transparent_blue if 'M1' in col else transparent_red for col in colnames_custom}
palette_normal = {col: blue if 'M1' in col else red for col in colnames_custom}

fig, ax = plt.subplots(1, 1, figsize=(12, 8))


box_plot = sns.boxplot(
    data=dataframe_reordered,
    palette=palette_transp,
    width=0.5,
    fliersize=0.5,
    linewidth=2.,
    ax=ax
)
ax.tick_params(axis='both', which='major', labelsize=14, width=2)
ax.tick_params(axis='both', which='minor', labelsize=14, width=2)
# for col in colnames_custom:
#     sns.scatterplot(
#         data=dataframe_reordered,
#         x=[col]*len(dataframe_reordered),
#         y=dataframe_reordered[col],
#         color=palette_normal[col],
#         marker='o',
#         s=40,
#         edgecolor='black',
#         linewidth=0.5,
#         alpha=0.5,
#     )

sns.despine()
# plt.xlabel('X-Axis Label', fontsize=14)
plt.ylabel('HbO concentration change', fontsize=16)

# Add y-axis legend

plt.axhline(y=0, color='black', linestyle='--')
plt.axvline(x=3.5, ymin=0, ymax=0.95, color='black')


x1, x2 = 2, 3   
y, h, col = 15.5, 0.5, 'k'
plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c=col)
plt.text((x1+x2)*.5, y+h, "**", ha='center', va='bottom', color=col, fontsize=14)


x1, x2 = 0, 2  
y, h, col = 16.5, 0.5, 'k'
plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c=col)
plt.text((x1+x2)*.5, y+h, "**", ha='center', va='bottom', color=col, fontsize=14)

x1, x2 = 4, 6  
y, h, col = 15., .5, 'k'
plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c=col)
plt.text((x1+x2)*.5, y+h, "*", ha='center', va='bottom', color=col, fontsize=14)

x1, x2 = 6, 7 
y, h, col = 16.25, 0.5, 'k'
plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c=col)
plt.text((x1+x2)*.5, y+h, "**", ha='center', va='bottom', color=col, fontsize=14)



plt.subplots_adjust(top=0.965,
bottom=0.09,
left=0.09,
right=0.965,
wspace=0.5)

# Show plot
plt.show()
fig.savefig('/home/avaldamon/Desktop/mean_pictures/BOXPLOX.PNG', dpi=300)


# plt.subplots_adjust(top=0.97,
# bottom=0.095,
# left=0.04,
# right=0.975,
# hspace=0.16,
# wspace=0.175)

In [160]:
plt.xticks()

(array([0. , 0.2, 0.4, 0.6, 0.8, 1. ]),
 [Text(0.0, 0, '0.0'),
  Text(0.2, 0, '0.2'),
  Text(0.4, 0, '0.4'),
  Text(0.6000000000000001, 0, '0.6'),
  Text(0.8, 0, '0.8'),
  Text(1.0, 0, '1.0')])

In [276]:

results_dict = {}
for i, col1 in enumerate(dataframe.columns):
    for j, col2 in enumerate(dataframe.columns):
        if i < j:
            ttest_result = pg.wilcoxon(dataframe[col1], dataframe[col2],)
            results_dict[(col1, col2)] = (ttest_result['W-val'].values[0], ttest_result['p-val'].values[0])

results_df = pd.DataFrame.from_dict(results_dict, orient='index', columns=['T-statistic', 'p-value'])

num_comparisons = len(results_df)
bonferroni_alpha = 0.05/12

results_df['significance'] = results_df['p-value'] < bonferroni_alpha
results_df['significance'] = results_df['significance'].apply(lambda x: 'Yes' if x else 'No')

print(results_df)


                T-statistic   p-value significance
(ME M1, ME S1)         14.0  0.006714           No
(ME M1, TS M1)         13.0  0.005371           No
(ME M1, TS S1)         20.0  0.021545           No
(ME M1, MI M1)         21.0  0.025574           No
(ME M1, MI S1)         23.0  0.035339           No
(ME M1, TI M1)         10.0  0.002625          Yes
(ME M1, TI S1)         24.0  0.041260           No
(ME S1, TS M1)         20.0  0.021545           No
(ME S1, TS S1)         36.0  0.187622           No
(ME S1, MI M1)         45.0  0.421204           No
(ME S1, MI S1)         38.0  0.229309           No
(ME S1, TI M1)         18.0  0.015076           No
(ME S1, TI S1)         47.0  0.488708           No
(TS M1, TS S1)         10.0  0.002625          Yes
(TS M1, MI M1)         21.0  0.025574           No
(TS M1, MI S1)         27.0  0.063721           No
(TS M1, TI M1)         42.0  0.330261           No
(TS M1, TI S1)         20.0  0.021545           No
(TS S1, MI M1)         42.0  0.

In [96]:
import pandas as pd
import pingouin as pg

# Sample DataFrame (replace this with your actual DataFrame)
# Assuming your DataFrame is named dataframe
# dataframe = ...

# Perform paired t-tests and store results in a dictionary
results_dict = {}
for i, col1 in enumerate(dataframe.columns):
    for j, col2 in enumerate(dataframe.columns):
        if i < j:  # Only perform test once for each unique pair
            ttest_result = pg.wilcoxon(dataframe[col1], dataframe[col2], paired=True)
            results_dict[(col1, col2)] = (ttest_result['T'].values[0], ttest_result['p-val'].values[0])

# Create a DataFrame from the results dictionary
results_df = pd.DataFrame.from_dict(results_dict, orient='index', columns=['T-statistic', 'p-value'])

# Calculate Benjamini-Hochberg corrected significance level
p_values = results_df['p-value'].values
_, bh_p_values = pg.multicomp(p_values, method='fdr_bh')

# Add "significance" column
results_df['significance'] = bh_p_values < 0.05
results_df['significance'] = results_df['significance'].apply(lambda x: 'Yes' if x else 'No')

print(results_df)


                T-statistic   p-value significance
(ME M1, ME S1)     2.371870  0.032575           No
(ME M1, TS M1)     3.246770  0.005850           No
(ME M1, TS S1)     3.109922  0.007680           No
(ME M1, MI M1)     2.528408  0.024105           No
(ME M1, MI S1)     3.370589  0.004573           No
(ME M1, TI M1)     2.994411  0.009658           No
(ME M1, TI S1)     2.460469  0.027484           No
(ME S1, TS M1)     2.253091  0.040815           No
(ME S1, TS S1)     1.857188  0.084445           No
(ME S1, MI M1)     1.410404  0.180251           No
(ME S1, MI S1)     2.233385  0.042359           No
(ME S1, TI M1)     2.348155  0.034083           No
(ME S1, TI S1)     1.680521  0.115022           No
(TS M1, TS S1)    -1.428765  0.175002           No
(TS M1, MI M1)    -1.206081  0.247774           No
(TS M1, MI S1)    -0.570694  0.577253           No
(TS M1, TI M1)    -0.039396  0.969131           No
(TS M1, TI S1)    -0.985604  0.341056           No
(TS S1, MI M1)    -0.504760  0.

In [ ]:
results_df

In [ ]:
m1 =  ['S9_D13 hbo',
  'S9_D18 hbo',
  'S10_D13 hbo',
  'S10_D16 hbo',
  'S10_D19 hbo',
  'S16_D13 hbo']

s1 = [['S13_D18 hbo',
  'S16_D16 hbo',
  'S16_D18 hbo',
  'S16_D19 hbo',
  'S16_D23 hbo',
  'S24_D19 hbo']]

ERROR:tornado.application:Exception in callback functools.partial(<function Kernel.enter_eventloop.<locals>.advance_eventloop at 0x7fd376387c10>)
Traceback (most recent call last):
  File "/home/avaldamon/.cache/pypoetry/virtualenvs/fnirs-pEil43oO-py3.8/lib/python3.8/site-packages/tornado/ioloop.py", line 740, in _run_callback
    ret = callback()
  File "/home/avaldamon/.cache/pypoetry/virtualenvs/fnirs-pEil43oO-py3.8/lib/python3.8/site-packages/ipykernel/kernelbase.py", line 455, in advance_eventloop
    eventloop(self)
  File "/home/avaldamon/.cache/pypoetry/virtualenvs/fnirs-pEil43oO-py3.8/lib/python3.8/site-packages/ipykernel/eventloops.py", line 147, in loop_qt5
    return loop_qt4(kernel)
  File "/home/avaldamon/.cache/pypoetry/virtualenvs/fnirs-pEil43oO-py3.8/lib/python3.8/site-packages/ipykernel/eventloops.py", line 121, in loop_qt4
    from IPython.external.qt_for_kernel import QtGui
  File "/home/avaldamon/.cache/pypoetry/virtualenvs/fnirs-pEil43oO-py3.8/lib/python3.8/site-p

In [180]:
porsche_colors = {
    'Black': '#0D0D0D',
    'White': '#F7F7F7',
    'Guards Red': '#C91111',
    'Carmine Red': '#960018',
    'Papaya': '#FFC173',
    'Racing Yellow': '#FFB400',
    'Speed Yellow': '#FFB400',
    'Signal Yellow': '#F5AB35',
    'Viper Green': '#2A7D4D',
    'Irish Green': '#00574C',
    'Gulf Blue': '#0087B8',
    'Miami Blue': '#22A0D4',
    'Sapphire Blue': '#004E7C',
    'Night Blue': '#0F3B5C',
    'Mexico Blue': '#00A1D6',
    'Riveria Blue': '#005D7E',
    'GT Silver Metallic': '#A0A0A0',
    'Agate Grey Metallic': '#828282',
    'Chalk': '#E8E8E8',
    'Crayon': '#A2A2A2',
    'Dolomite Silver Metallic': '#7A7A7A',
    'Jet Black Metallic': '#1D1D1D',
    'Mahogany Metallic': '#672023',
    'Moonlight Blue Metallic': '#202A44',
    'Night Blue Metallic': '#0F3B5C',
    'Quartzite Grey Metallic': '#717171',
    'Rhodium Silver Metallic': '#A0A0A0',
    'Sapphire Blue Metallic': '#004E7C',
    'Vulcano Grey Metallic': '#7E7E7E',
    'Crayon Metallic': '#A2A2A2',
    'Python Green': '#007C59',
    'Lava Orange': '#FF4E09',
    'Lizard Green': '#5D8B35',
    'Mamba Green': '#0C6E3D',
    'Mint Green': '#78A086',
    'Riviera Blue': '#005D7E'
}

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.patches import Rectangle
import math

fig, ax = plt.subplots()

# Iterate through the dictionary and plot each color
for i, (color_name, color_code) in enumerate(porsche_colors.items()):
    # Create a rectangle (square) for each color
    square = Rectangle((i, 0), 1, 1, facecolor=color_code, edgecolor='black')
    ax.add_patch(square)
    
    # Add a label below the square
    ax.text(i, -0.1, color_name, ha='center', va='center', fontsize=10)

# Set the limits of the plot
ax.set_xlim(0, len(porsche_colors))
ax.set_ylim(-0.5, 1)

# Hide the axes
ax.axis('off')

# Display the plot
plt.show()

In [195]:
filename = '/mnt/diskus/fNIRS data ME_MI_TS_TI_SA/AA/AA_ME'
raw_intensity = mne.io.read_raw_nirx(filename, verbose=False, preload=True)

raw_od = optical_density(raw_intensity) #from row wavelength data
# raw_od.apply_function(detrender, picks=raw_od.ch_names)
raw_od_shorts = mne_nirs.channels.get_short_channels(raw_od)
raw_od.drop_channels(DROP_CHANS) #we had a non-existent channel
raw_od = raw_od.filter(**FILTER_DICT)
raw_od = mne_nirs.signal_enhancement.short_channel_regression(raw_od)
raw_od = mne_nirs.channels.get_long_channels(raw_od)
raw_od = temporal_derivative_distribution_repair(raw_od)

raw_od_unfiltered = raw_od.copy() #repairs movement artifacts

# ## BAD CHANNELS ###
# channel_std = np.abs(np.std(raw_od.copy().pick([i for i in raw_od.ch_names if '760' in i]).get_data(), 
#                     axis=(1)))
# # print(channel_std)
# biggest_indices = np.argpartition(channel_std, -20)[-20:]
# smallest_indices = np.argpartition(channel_std,  5)[:5]
# smallest_indices_list = smallest_indices.tolist()
# biggest_indices_list = biggest_indices.tolist()
# deviant_channels_idx = biggest_indices_list + smallest_indices_list

# bad_channels_hbo = [raw_od.copy().pick([i for i in raw_od.ch_names if '760' in i]).ch_names[i]
#             for i in deviant_channels_idx]
# bad_channels_hbr = [i.replace('760', '850') for i in bad_channels_hbo]
# bad_channels = bad_channels_hbo + bad_channels_hbr
# raw_od.info['bads'] = bad_channels
# raw_od.interpolate_bads()


# raw_od.apply_function(rolling_mean_filter, picks=raw_od.ch_names)


raw_haemo = beer_lambert_law(raw_od, ppf=0.1) #from wavelength to HbO\HbR
raw_haemo = enhance_negative_correlation(raw_haemo)

In [196]:
# raw_haemo.pick(M1_LEFT_ROI_HBO).info.save('/mnt/diskus/infos/M1_info.fif')
# raw_haemo.pick(S1_LEFT_ROI_HBO).info.save('/mnt/diskus/infos/S1_info.fif')
raw_haemo.pick(REMAINING_LEFT_ROI_HBO).info.save('/mnt/diskus/infos/BETWEEN_info.fif')